### Sequences Alignment

#### 1st approach: alignment based on sequences distance

- Hamming
- Levenshtein

---
#### 2nd approach: recursive alignment using sequences distance

In [1]:
function hamming(x,y)
    if length(x) != length(y)
        println("ERROR: sequences should have equal lenghts!")
        return
    else
        d = 0
        for i = 1:length(x)
            if x[i] != y[i]
                d = d + 1
            end
        end
        return d
    end
end

hammington

In [2]:
x = "wheeaaa"; y = "ghearpa"
d = hamming(x,y)

4

How to read real sequences from online database

In [4]:
using FastaIO
#using Images
using OffsetArrays
using PyPlot
using DelimitedFiles
using BenchmarkTools
using StatsBase
using LinearAlgebra
using Printf
using HTTP

In [7]:
function sequenceDownload(sequence)

    sequenceFile = sequence * ".fasta"

    URL = "https://www.uniprot.org/uniprotkb/" * sequenceFile

    query = HTTP.get(URL)
    fastaString=String(query.body)

    open(sequenceFile,"w") do f
        write(f,fastaString)
    end

    FastaIO.readfasta(sequenceFile)[1][2]
end

sequenceDownload (generic function with 1 method)

In [8]:
HBB_Human = sequenceDownload("P68871")
HBA_Bonobo = sequenceDownload("P69906")
HBA_Chimp = sequenceDownload("P69907")
HBA_Donkey = sequenceDownload("P01959")
LegHem = sequenceDownload("P02240")

"MGALTESQAALVKSSWEEFNANIPKHTHRFFILVLEIAPAAKDLFSFLKGTSEVPQNNPELQAHAGKVFKLVYEAAIQLQVTGVVVTDATLKNLGSVHVSKGVADAHFPVVKEAILKTIKEVVGAKWSEELNSAWTIAYDELAIVIKKEMNDAA"

In [39]:
d = hamming(HBB_Human,HBA_Bonobo)

0